# AdventureWorks Data Exploration

This notebook provides initial exploration of the AdventureWorks dataset.

## Datasets Available:
- **Sales Data**: 56,046 transactions (2015-2017)
- **Customers**: 18,148 customers
- **Products**: 293 products
- **Returns**: 1,809 return records
- **Territories**: 10 sales territories
- **Product Categories & Subcategories**: Product hierarchy

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 1. Load Datasets

In [ ]:
# Load merged sales data
sales = pd.read_csv('../data/processed/AdventureWorks_Sales_Merged.csv', parse_dates=['OrderDate', 'StockDate'])

# Load other datasets
customers = pd.read_csv('../data/raw/converted_AdventureWorks_Customers.csv')
products = pd.read_csv('../data/raw/AdventureWorks_Products.csv')
returns = pd.read_csv('../data/raw/converted_AdventureWorks_Returns.csv', parse_dates=['ReturnDate'])
territories = pd.read_csv('../data/raw/AdventureWorks_Territories.csv')
categories = pd.read_csv('../data/raw/AdventureWorks_Product_Categories.csv')
subcategories = pd.read_csv('../data/raw/AdventureWorks_Product_Subcategories.csv')

print(f"Sales: {sales.shape}")
print(f"Customers: {customers.shape}")
print(f"Products: {products.shape}")
print(f"Returns: {returns.shape}")
print(f"Territories: {territories.shape}")

## 2. Sales Data Overview

In [ ]:
# Basic info
print("Sales Data Info:")
print(sales.info())
print("\nFirst few rows:")
sales.head()

In [ ]:
# Summary statistics
print("Sales Summary Statistics:")
sales.describe()

In [ ]:
# Check for missing values
print("Missing Values:")
print(sales.isnull().sum())
print(f"\nTotal missing: {sales.isnull().sum().sum()}")

## 3. Enrich Sales with Product Information

In [ ]:
# Merge products with subcategories and categories
products_full = products.merge(subcategories, on='ProductSubcategoryKey', how='left')
products_full = products_full.merge(categories, on='ProductCategoryKey', how='left')

# Merge sales with enriched products
sales_enriched = sales.merge(products_full, on='ProductKey', how='left')
sales_enriched = sales_enriched.merge(territories, left_on='TerritoryKey', right_on='SalesTerritoryKey', how='left')

# Calculate revenue
sales_enriched['Revenue'] = sales_enriched['OrderQuantity'] * sales_enriched['ProductPrice']
sales_enriched['Profit'] = sales_enriched['Revenue'] - (sales_enriched['OrderQuantity'] * sales_enriched['ProductCost'])

print(f"Enriched Sales Shape: {sales_enriched.shape}")
print(f"\nNew columns: {[col for col in sales_enriched.columns if col not in sales.columns]}")
sales_enriched.head(3)

## 4. Key Business Metrics

In [ ]:
print("="*70)
print("KEY BUSINESS METRICS")
print("="*70)

print(f"\nTotal Revenue: ${sales_enriched['Revenue'].sum():,.2f}")
print(f"Total Profit: ${sales_enriched['Profit'].sum():,.2f}")
print(f"Profit Margin: {(sales_enriched['Profit'].sum() / sales_enriched['Revenue'].sum() * 100):.2f}%")

print(f"\nTotal Orders: {sales_enriched['OrderNumber'].nunique():,}")
print(f"Total Customers: {sales_enriched['CustomerKey'].nunique():,}")
print(f"Total Products Sold: {sales_enriched['ProductKey'].nunique():,}")

print(f"\nAverage Order Value: ${sales_enriched.groupby('OrderNumber')['Revenue'].sum().mean():,.2f}")
print(f"Average Items per Order: {sales_enriched.groupby('OrderNumber')['OrderQuantity'].sum().mean():.2f}")

print(f"\nTotal Returns: {returns.shape[0]:,}")
print(f"Return Rate: {(returns['ReturnQuantity'].sum() / sales_enriched['OrderQuantity'].sum() * 100):.2f}%")

## 5. Sales Trends Over Time

In [ ]:
# Monthly sales trend
sales_enriched['YearMonth'] = sales_enriched['OrderDate'].dt.to_period('M')
monthly_sales = sales_enriched.groupby('YearMonth').agg({
    'Revenue': 'sum',
    'Profit': 'sum',
    'OrderNumber': 'nunique',
    'OrderQuantity': 'sum'
}).reset_index()

monthly_sales['YearMonth'] = monthly_sales['YearMonth'].astype(str)

# Plot revenue trend
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Revenue
axes[0, 0].plot(monthly_sales['YearMonth'], monthly_sales['Revenue'], marker='o', linewidth=2)
axes[0, 0].set_title('Monthly Revenue Trend', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Month')
axes[0, 0].set_ylabel('Revenue ($)')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# Profit
axes[0, 1].plot(monthly_sales['YearMonth'], monthly_sales['Profit'], marker='o', color='green', linewidth=2)
axes[0, 1].set_title('Monthly Profit Trend', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Profit ($)')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# Orders
axes[1, 0].plot(monthly_sales['YearMonth'], monthly_sales['OrderNumber'], marker='o', color='orange', linewidth=2)
axes[1, 0].set_title('Monthly Order Count', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Number of Orders')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Quantity
axes[1, 1].plot(monthly_sales['YearMonth'], monthly_sales['OrderQuantity'], marker='o', color='red', linewidth=2)
axes[1, 1].set_title('Monthly Quantity Sold', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Month')
axes[1, 1].set_ylabel('Quantity')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/visualizations/sales_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Sales trends chart saved!")

## 6. Category Analysis

In [ ]:
# Revenue by category
category_performance = sales_enriched.groupby('CategoryName').agg({
    'Revenue': 'sum',
    'Profit': 'sum',
    'OrderQuantity': 'sum',
    'OrderNumber': 'nunique'
}).round(2)

category_performance['Profit_Margin_%'] = (category_performance['Profit'] / category_performance['Revenue'] * 100).round(2)
category_performance = category_performance.sort_values('Revenue', ascending=False)

print("Category Performance:")
print(category_performance)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Revenue by category
category_performance['Revenue'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Revenue by Product Category', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Revenue ($)')
axes[0].set_xlabel('')
axes[0].tick_params(axis='x', rotation=45)

# Profit margin by category
category_performance['Profit_Margin_%'].plot(kind='bar', ax=axes[1], color='green')
axes[1].set_title('Profit Margin by Product Category', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Profit Margin (%)')
axes[1].set_xlabel('')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../outputs/visualizations/category_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Geographic Analysis

In [ ]:
# Revenue by territory
territory_performance = sales_enriched.groupby(['Country', 'Region']).agg({
    'Revenue': 'sum',
    'Profit': 'sum',
    'OrderNumber': 'nunique',
    'CustomerKey': 'nunique'
}).round(2)

territory_performance = territory_performance.sort_values('Revenue', ascending=False)

print("Territory Performance:")
print(territory_performance)

# Visualize top territories
top_territories = sales_enriched.groupby('Region')['Revenue'].sum().sort_values(ascending=True).tail(10)

plt.figure(figsize=(12, 6))
top_territories.plot(kind='barh', color='coral')
plt.title('Revenue by Territory (Top 10)', fontsize=14, fontweight='bold')
plt.xlabel('Revenue ($)')
plt.ylabel('Region')
plt.tight_layout()
plt.savefig('../outputs/visualizations/territory_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Customer Analysis

In [ ]:
# Customer demographics
print("Customer Demographics:")
print("\nGender Distribution:")
print(customers['Gender'].value_counts())

print("\nMarital Status:")
print(customers['MaritalStatus'].value_counts())

print("\nEducation Level:")
print(customers['EducationLevel'].value_counts())

print("\nOccupation:")
print(customers['Occupation'].value_counts())

print("\nHome Ownership:")
print(customers['HomeOwner'].value_counts())

In [ ]:
# Customer purchase behavior
customer_behavior = sales_enriched.groupby('CustomerKey').agg({
    'Revenue': 'sum',
    'OrderNumber': 'nunique',
    'OrderQuantity': 'sum',
    'OrderDate': ['min', 'max']
}).reset_index()

customer_behavior.columns = ['CustomerKey', 'TotalRevenue', 'OrderCount', 'TotalQuantity', 'FirstPurchase', 'LastPurchase']
customer_behavior['AvgOrderValue'] = customer_behavior['TotalRevenue'] / customer_behavior['OrderCount']

print("Customer Behavior Summary:")
print(customer_behavior.describe())

# Top customers
print("\nTop 10 Customers by Revenue:")
print(customer_behavior.nlargest(10, 'TotalRevenue')[['CustomerKey', 'TotalRevenue', 'OrderCount', 'AvgOrderValue']])

## 9. Product Analysis

In [ ]:
# Top products
product_performance = sales_enriched.groupby(['ProductName', 'CategoryName']).agg({
    'Revenue': 'sum',
    'OrderQuantity': 'sum',
    'Profit': 'sum'
}).reset_index()

product_performance = product_performance.sort_values('Revenue', ascending=False)

print("Top 20 Products by Revenue:")
print(product_performance.head(20))

# Visualize top products
top_products = product_performance.head(10)

plt.figure(figsize=(12, 8))
plt.barh(range(len(top_products)), top_products['Revenue'], color='teal')
plt.yticks(range(len(top_products)), top_products['ProductName'], fontsize=10)
plt.xlabel('Revenue ($)', fontsize=12)
plt.title('Top 10 Products by Revenue', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../outputs/visualizations/top_products.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. Returns Analysis

In [ ]:
# Merge returns with products
returns_enriched = returns.merge(products_full, on='ProductKey', how='left')

# Return analysis
print("Returns Overview:")
print(f"Total Returns: {returns['ReturnQuantity'].sum():,}")
print(f"Total Orders Returned: {returns.shape[0]:,}")
print(f"\nReturn Rate: {(returns['ReturnQuantity'].sum() / sales_enriched['OrderQuantity'].sum() * 100):.2f}%")

# Returns by category
returns_by_category = returns_enriched.groupby('CategoryName')['ReturnQuantity'].sum().sort_values(ascending=False)
print("\nReturns by Category:")
print(returns_by_category)

# Calculate return rate by product
product_sales = sales_enriched.groupby('ProductKey')['OrderQuantity'].sum()
product_returns = returns.groupby('ProductKey')['ReturnQuantity'].sum()
return_rate_df = pd.DataFrame({
    'Sales': product_sales,
    'Returns': product_returns
}).fillna(0)
return_rate_df['ReturnRate_%'] = (return_rate_df['Returns'] / return_rate_df['Sales'] * 100).round(2)
return_rate_df = return_rate_df.sort_values('ReturnRate_%', ascending=False)

print("\nProducts with Highest Return Rates:")
print(return_rate_df.head(10))

## 11. Save Enriched Dataset

In [ ]:
# Save enriched sales data for modeling
sales_enriched.to_csv('../data/processed/AdventureWorks_Sales_Enriched.csv', index=False)
print("✓ Enriched sales data saved to: data/processed/AdventureWorks_Sales_Enriched.csv")

# Save customer behavior data
customer_behavior.to_csv('../data/processed/AdventureWorks_Customer_Behavior.csv', index=False)
print("✓ Customer behavior data saved to: data/processed/AdventureWorks_Customer_Behavior.csv")

print("\nData exploration complete!")

## Summary

### Key Findings:
1. **Sales Growth**: Strong growth from 2015 to 2017
2. **Product Categories**: Identify which categories drive the most revenue
3. **Geographic Distribution**: Territory performance varies significantly
4. **Customer Behavior**: Understand purchase patterns and customer value
5. **Returns**: Monitor return rates by product and category

### Next Steps:
1. Build revenue forecasting models
2. Develop churn prediction model
3. Create return risk scoring system
4. Customer segmentation analysis